In [ ]:
# Importeer benodigde modules
from tqdm.notebook import tqdm
from itertools import product
from ultralytics import YOLO
import ipywidgets as widgets
from PIL import Image
from onnx import load
import logging
import shutil
import os

# Hele kaart Predictie

Dit script is bedoeld om voorspellingen te doen op volledige foto's die normaal gesproken gesegmenteerd worden voordat het model ze ziet. Foto's worden gesegmenteerd, geannoteerd door het model, en de labels worden samengevoegd tot één groot label per foto. De gebruiker wordt gevraagd om de verwachte grootte, oriëntatie en lengte van de originele foto's op te geven om onverwachte en soms onzichtbare rotaties in foto's te neutraliseren.

Na het genereren van labels is het van belang dat de output gecontroleerd wordt, met een nadruk op de naden/randen van de segmenten. Als bij het genereren een overlap groter dan 0 is opgegeven, is de kans op overlappende annotaties vergroot en zal hierop gecontroleerd moeten worden.
De gegenereerde segmenten en bijbehorende labels staan in de map "Tijdelijke data". Mocht een gebruiker interesse hebben in deze data, dan moet deze alle bestanden kopiëren naar een nieuwe locatie voordat het script opnieuw gedraaid wordt.

**LET OP!** Bij het draaien van het script worden naast de tijdelijke bestanden ook de opgegeven OutputLocatie opgeruimd, waarbij alle bestanden in deze mappen verwijderd worden.

---
<blockquote style="border-left: 5px solid red; padding-left: 10px;">

## **Attentie!** 
<code> Update onderstaande cell voordat je de het script runt </code></blockquote>
 

In [ ]:
# Input map met de foto's die Labels nodig hebben.
InputFotos : str = r"pad/naar/jouw/folder"
# Bijvoorbeeld: r"pad/naar/jouw/folder"

# Output map waar de hernoemde labels en de  resized foto's worden opgeslagen.
OutputLocatie : str = r"folder"
# Bijvoorbeeld: r"pad/naar/jouw/output/folder"

# Locatie van het model dat gebruikt wordt (Moet .onnx of .pt zijn).
ModelLocatie : str = r"pad/naar/jouw/model/bestand.pt"
# Bijvoorbeeld: r"pad/naar/jouw/model/bestand.pt"

# Bepaal of de hele foto horizontaal(True) of verticaal(False) is.
HeleFotoHorizontaal : bool = True
# Bijvoorbeeld: True OF False

# Bepaal of de rotatie naar rechts(True) of naar links(False) is.
RotatieNaarRechts : bool = False
# Bijvoorbeeld: True OF False

# Geef de langste zijde van de hele foto in pixels.
HeleFotoLangsteZijde : int = 3200
# Bijvoorbeeld: 3200 OF 1080 OF 720 etc.

# Geef lengte van de zijde van het vierkanter segment in pixels.
SegmentGrote : int = 640
# Bijvoorbeeld: 640 OF 320 OF 160 etc.

# Het percentage overlap tussen segmenten in procenten.
OverlapInProcent : float = 0
# Bijvoorbeeld: 0 OF 10 OF 25.2 etc.

# De confidence threshold in procenten.
ConfidenceInProcenten : float = 5
# Bijvoorbeeld: 5 OF 10 OF 25.2 etc.

---
## **Code**

In [ ]:
# Onderdrukt de printstatements van yolo
logging.getLogger('ultralytics').setLevel(logging.WARNING)

#maakt een output scherm aan
OutputScreen = widgets.Output()
display(OutputScreen)

#ondersteunde bestandstypen (Lowercase)
ImageTypes = [".jpg", ".png", "jpeg"]

# Zet de variabelen om naar correcte eenheden
OverlapInPixels = int(SegmentGrote * (OverlapInProcent / 100))
ConfidenceInFractie = ConfidenceInProcenten / 100

# Controle/aanmaak van output mappen
TileFotos = os.path.join(os.getcwd(), "Tijdelijke data\\Output\\images")
TileLabels = os.path.join(os.getcwd(), "Tijdelijke data\\Output\\labels")
OutputLocatieLabels = os.path.join(os.getcwd(), OutputLocatie, "labels")
OutputLocatieFotos = os.path.join(os.getcwd(), OutputLocatie, "images")

for path in [TileLabels, TileFotos, OutputLocatieLabels, OutputLocatieFotos]:
    if os.path.exists(path):
        shutil.rmtree(path)
    os.makedirs(path)

# maakt een lijst van alle valide bestanden in de input map
LijstFotos = [file for file in os.listdir(InputFotos) if file.lower().endswith(tuple(ImageTypes))]

# Bepaal de rotatiehoek op basis van RotatieNaarRechts
angle = (lambda: 90 if RotatieNaarRechts else 270)()

with OutputScreen:
    # loopt door alle valide bestanden en maakt segmenten
    for Files in tqdm(LijstFotos, desc="1/3 Segmenting files"):
        with Image.open(os.path.join(InputFotos, Files)) as image:

            Width, Height = image.size

            # Checkt of de foto gedraaid moet worden
            if Width > Height:
                if HeleFotoHorizontaal:
                    image = image.rotate(angle, expand=True)
                    Width, Height = image.size
            elif Height > Width:
                if not HeleFotoHorizontaal:
                    image = image.rotate(angle, expand=True)
                    Width, Height = image.size

            # Berekend de nieuwe hoogte van de foto zodat verhoudingen niet veranderd worden
            if Width>Height and Width != HeleFotoLangsteZijde:
                NewHeight = int(HeleFotoLangsteZijde/Width*Height)

                image = image.resize((HeleFotoLangsteZijde, NewHeight))
                Width, Height = image.size

            elif Width <= Height and Height != HeleFotoLangsteZijde:
                NewWidth = int(HeleFotoLangsteZijde/Height*Width)

                image = image.resize((NewWidth, HeleFotoLangsteZijde))
                Width, Height = image.size

            # Slaat de hele foto op zodat heventuele onzichtbare draaiingen niet geen probleem vormen
            image.save(os.path.join(OutputLocatieFotos, Files))

            # Maakt segmenten van de foto
            FileName, FileType = os.path.splitext(Files)
            grid = product(range(0, Width, (SegmentGrote-OverlapInPixels)), range(0, Height, (SegmentGrote-OverlapInPixels)))

            for x, y in grid:
                box = (x, y, x+SegmentGrote, y+SegmentGrote)
                # y en x geven de locatie van de rechter boven hoek van het segment op de hele foto voor reconstructie
                # de volgorde is y daarna x om consistent te zijn met de AiAssist-AnnotatieTool van FishingBehindTheNet
                out = os.path.join(TileFotos, f'{FileName}_{y}_{x}{FileType}') 
                image.crop(box).save(out)




# laat het geselecteerde model in en filtert de labelnamen eruit.
if ModelLocatie.endswith(".pt"):
    model = YOLO(ModelLocatie, task="detect")
    MLabels = model.model.names
    ModelLabels = ""
    for labels in MLabels:
        ModelLabels += f"{MLabels[labels]}<br>"
elif ModelLocatie.endswith(".onnx"):
    model = load(ModelLocatie)
    MLabels = (
        str(model)
        .split('\nmetadata_props {\n  key: "names"\n  value: "')[-1]
        .replace("{", "")
        .replace("}", "")
        .replace("\\'", "")
        .replace("\n", "")
        .replace('"', "")
        .split(", ")
    )
    ModelLabels = ""
    for labels in MLabels:
        ModelLabels += f"{labels.split(': ')[-1]}<br>"

    model = YOLO(ModelLocatie, task="detect")

# slaat de labels van het model op in een txt bestand (YOLO format)
LabelDoc = OutputLocatieLabels + "\\Labels.txt"
with open(LabelDoc, "w+") as e:
    e.write("")
    with open(LabelDoc, "a") as f:
        MLabels = ModelLabels.split("<br>")
        for labels in MLabels:
            if labels:
                f.write(f"{labels}\n")

# Maakt een lijst van alle segmenten waar nog geen labels voor zijn
TileFiles = [f for f in os.listdir(TileFotos) if f.lower().endswith(tuple(ImageTypes))]
TileFiles = [f for f in TileFiles if not os.path.exists(os.path.join(TileLabels, (f.rsplit(".", 1)[0] + ".txt")))]

# Voert de voorspellingen uit op de segmenten
with OutputScreen:
    for tile in tqdm(TileFiles, desc="2/3 Running predictions"):
        tile_path = os.path.join(TileFotos, tile)
        model.predict(tile_path, save_txt=True, conf=ConfidenceInFractie, exist_ok=True, project="Tijdelijke data", name = "Output")



LabelSegmenten = os.listdir(TileLabels)

with OutputScreen:
    # loop over alle resided input foto's
    for Foto in tqdm(os.listdir(OutputLocatieFotos), desc="3/3 Compiling labels"):
        FotoName, FotoType = os.path.splitext(Foto)
        # extra check op dwaalgasten in de output map
        if FotoType.lower() in ImageTypes:
            LocatieHeelLabel = os.path.join(OutputLocatieLabels, FotoName + ".txt")
            Labels = []
            # filter alle label segmenten uit die bij de foto horen
            for Label in LabelSegmenten:
                if Label.startswith(FotoName):
                    Labels.append(Label)

            with Image.open(os.path.join(OutputLocatieFotos, Foto)) as img:
                img_x, img_y = img.size

                # checkt of maakt het label bestand voor de hele foto aan en verzekert dat het leeg is
                with open(LocatieHeelLabel, "w+") as a:
                    a.write("")

                for Label in Labels:
                    # Verzameld de benodigde pathes
                    LabelName, LabelType = os.path.splitext(Label)
                    SegmentName = LabelName + FotoType
                    SegmentLocation = os.path.join(TileFotos, SegmentName)
                    LabelLocation = os.path.join(TileLabels, Label)

                    if os.path.exists(SegmentLocation):
                        # extraheert de locatie van het segment op de hele foto uit de bestandsnaam
                        _, Ymin, Xmin = LabelName.rsplit("_", 2)
                        with open(LabelLocation, "r")as b:
                            Annotaties = b.readlines()
                            # leest de YOLO annotaties uit en rekent ze om naar de nieuwe coördinaten 
                            for bbox in Annotaties:
                                Annotaties_Parts = bbox.split(" ")
                                # YOLO Structuur: <class_id> <x_center> <y_center> <width> <height>
                                    # (alle waarde, behalve class_id, tussen 0 en 1)
                                BoxLabel = Annotaties_Parts[0]
                                BboxX = (float(Annotaties_Parts[1]) * SegmentGrote + float(Xmin))/img_x
                                BboxY = (float(Annotaties_Parts[2]) * SegmentGrote + float(Ymin))/img_y
                                BboxW = (float(Annotaties_Parts[3]) * SegmentGrote)/img_x
                                BboxH = (float(Annotaties_Parts[4]) * SegmentGrote)/img_y
                                with open(LocatieHeelLabel, "+a") as c:
                                    c.write(f"{BoxLabel} {BboxX} {BboxY} {BboxW} {BboxH}\n")

